In [36]:
from langgraph.graph import StateGraph,START,END
from langchain_openai import ChatOpenAI
from typing import TypedDict,Annotated
from dotenv import load_dotenv
from pydantic import BaseModel,Field
import operator

In [37]:

load_dotenv() 
model = ChatOpenAI(model='gpt-4o-mini') # this model gives structured op
# Concepts discussed : llel , structured_op , reducer  

In [38]:
class EvaluationSchema(BaseModel):
    feedback: str = Field(description='Detailed feedback for the essay') 
    score: int = Field(description='Score out of 10',ge=0,le=10)

In [39]:
structured_model = model.with_structured_output(EvaluationSchema)

In [40]:
essay="""India is strategically positioned as the "AI Garage of the World." Its role in the global AI landscape is defined by three pillars: Democratization, Talent, and Ethics.

With the world’s largest developer base and a thriving startup ecosystem, India provides the scale needed to test AI solutions in diverse sectors like agriculture, healthcare, and education. The government’s "AI for All" initiative emphasizes inclusive technology, ensuring AI bridges the digital divide rather than widening it.

Furthermore, India acts as a "bridge" between the Global North and South, advocating for Sovereign AI and ethical frameworks that protect data privacy while fostering innovation. By focusing on frugal, impactful AI, India is transforming from a back-office service provider into a global leader in foundational AI research and deployment."""

In [41]:
prompt=f"Evaluate the quality of following essay and provide a feedback and assign a score out of 10 :\n {essay}"
structured_model.invoke(prompt)

EvaluationSchema(feedback='The essay effectively captures India\'s growing significance in the AI sector by structuring its arguments around three key pillars: Democratization, Talent, and Ethics. The overall strengths include the clear and concise presentation of ideas, highlighting India\'s large developer base and startup ecosystem, along with initiatives like "AI for All" that promote inclusivity. Additionally, the mention of India\'s role in bridging the Global North and South and advocating for ethical AI adds depth to the argument. However, the essay could benefit from more elaborate examples or case studies to support the claims made about the impact of AI in various sectors. Expanding on how India plans to overcome challenges related to AI implementation would also strengthen the analysis. Overall, it is a solid and informative piece but could be enhanced by a deeper exploration of the topics introduced. The writing is coherent and flows well, but it lacks some depth and illus

In [42]:
class UPSCState(TypedDict): 

    essay:str 
    language_feedback: str 
    analysis_feedback: str 
    clarity_feedback: str 
    overall_feedback: str 
    individual_scores: Annotated[list[int],operator.add] # the reducer
    final_evaluation: float
    avg_score: float

In [43]:
def evaluate_language(stae: UPSCState):
    prompt=f"Evaluate the quality of following essay and provide a feedback and assign a score out of 10 :\n {essay}"
    output = structured_model.invoke(prompt)

    return {'language_feedback': output.feedback,'individual_scores':[output.score]}

In [44]:
def evaluate_analysis(state: UPSCState): 
    prompt=f"Evaluate the depth of analysis of the following essay and provide a feedback and assign a score out of 10 :\n {essay}"
    output = structured_model.invoke(prompt)

    return {'analysis_feedback': output.feedback,'individual_scores':[output.score]}

In [45]:
def evaluate_thought(state: UPSCState): 
    prompt = f"Evaluate the clarity of feedback of the following essay and provide a feedback and assign a score out of 10 :\n {essay}"
    output = structured_model.invoke(prompt)

    return {'clarity_feedback': output.feedback,'individual_scores':[output.score]}

In [46]:
def final_evaluation(state: UPSCState): 

    prompt = f"Based on the given feedbacks cretae a summarised feedback \n language feedback -  {state['language_feedback']} \n depth of analysis feedback : {state['analysis_feedback']} \n clarity of thought feedback - {state['clarity_feedback']}"
    output = structured_model.invoke(prompt)

    avg_score = sum(state['individual_scores'])/len(state['individual_scores'])

    return {'overall_feedback':output,'avg_score':avg_score}

In [47]:
graph = StateGraph(UPSCState) 

graph.add_node('evaluate_language',evaluate_language)
graph.add_node('evaluate_analysis',evaluate_analysis)
graph.add_node('evaluate_thought',evaluate_thought)
graph.add_node('final_evaluation',final_evaluation)

graph.add_edge(START,'evaluate_language')
graph.add_edge(START,'evaluate_analysis')
graph.add_edge(START,'evaluate_thought')

graph.add_edge('evaluate_language','final_evaluation')
graph.add_edge('evaluate_analysis','final_evaluation')
graph.add_edge('evaluate_thought','final_evaluation')

graph.add_edge('final_evaluation',END) 
app=graph.compile()

In [48]:
initial_state = {
    'essay':essay
}
final_state=app.invoke(initial_state) 
print(final_state)

{'essay': 'India is strategically positioned as the "AI Garage of the World." Its role in the global AI landscape is defined by three pillars: Democratization, Talent, and Ethics.\n\nWith the world’s largest developer base and a thriving startup ecosystem, India provides the scale needed to test AI solutions in diverse sectors like agriculture, healthcare, and education. The government’s "AI for All" initiative emphasizes inclusive technology, ensuring AI bridges the digital divide rather than widening it.\n\nFurthermore, India acts as a "bridge" between the Global North and South, advocating for Sovereign AI and ethical frameworks that protect data privacy while fostering innovation. By focusing on frugal, impactful AI, India is transforming from a back-office service provider into a global leader in foundational AI research and deployment.', 'language_feedback': "The essay presents a clear and focused argument on India's role in the global AI landscape. It effectively outlines the th